In [1]:
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

In [10]:
# Pergunta o nome do movimento
movimento = input("Digite o nome do movimento (ex.: Abrir_Mao, Fechar_Mao, Flexao, Extensao): ")


Digite o nome do movimento (ex.: Abrir_Mao, Fechar_Mao, Flexao, Extensao): Abrir_punho


In [ ]:
# Upload dos arquivos CSV
uploaded = files.upload()

In [13]:
# Pastas de saída
pasta_base = "/content/Dados_Filtrados"
pasta_csv_base = os.path.join(pasta_base, movimento, "CSV_Filtrados")
pasta_graficos_base = os.path.join(pasta_base, movimento, "Graficos")

In [14]:
# Cria pastas de janelas
janelas = [5, 10, 20]
for janela in janelas:
    os.makedirs(os.path.join(pasta_csv_base, f"Janela_{janela}"), exist_ok=True)
    os.makedirs(os.path.join(pasta_graficos_base, f"Janela_{janela}"), exist_ok=True)

In [15]:
# Função para filtrar e salvar
def aplicar_filtro_e_salvar(nome_arquivo, janela):
    # Carrega CSV
    dados = pd.read_csv(nome_arquivo)

    # Checa nome das colunas
    if "Tempo (ms)" in dados.columns:
        tempo_col = "Tempo (ms)"
    elif "Tempo" in dados.columns:
        tempo_col = "Tempo"
    else:
        raise KeyError("Coluna de tempo não encontrada (esperado: 'Tempo (ms)' ou 'Tempo')")

    # Aplica filtro de média móvel
    dados['Valor1_filtrado'] = dados['Valor1'].rolling(window=janela, center=True, min_periods=1).mean()
    dados['Valor2_filtrado'] = dados['Valor2'].rolling(window=janela, center=True, min_periods=1).mean()

    # Salva CSV filtrado
    nome_base = os.path.splitext(os.path.basename(nome_arquivo))[0]
    caminho_csv_filtrado = os.path.join(pasta_csv_base, f"Janela_{janela}", f"{nome_base}_filtrado_{janela}.csv")
    dados_filtrados = dados[[tempo_col, 'Valor1_filtrado', 'Valor2_filtrado']]
    dados_filtrados.to_csv(caminho_csv_filtrado, index=False)

    # Plota gráfico com escala fixa
    plt.figure(figsize=(10, 6))

    # Sinal 1
    plt.subplot(2, 1, 1)
    plt.plot(dados[tempo_col], dados['Valor1'], label='Original', alpha=0.4)
    plt.plot(dados[tempo_col], dados['Valor1_filtrado'], label=f'Filtrado {janela}', color='blue')
    plt.title('Sinal 1')
    plt.ylabel('Amplitude')
    plt.ylim(0, 4000)  # Escala fixa Y
    plt.xlim(0, 2000)  # Escala fixa X
    plt.legend()

    # Sinal 2
    plt.subplot(2, 1, 2)
    plt.plot(dados[tempo_col], dados['Valor2'], label='Original', alpha=0.4)
    plt.plot(dados[tempo_col], dados['Valor2_filtrado'], label=f'Filtrado {janela}', color='green')
    plt.title('Sinal 2')
    plt.xlabel('Tempo (ms)')
    plt.ylabel('Amplitude')
    plt.ylim(0, 4000)  # Escala fixa Y
    plt.xlim(0, 2000)  # Escala fixa X
    plt.legend()

    plt.tight_layout()

    # Salva PNG
    caminho_imagem = os.path.join(pasta_graficos_base, f"Janela_{janela}", f"{nome_base}_grafico_{janela}.png")
    plt.savefig(caminho_imagem)
    plt.close()

In [16]:
# Aplica para todos os arquivos e janelas
for nome_arquivo in uploaded.keys():
    for janela in janelas:
        aplicar_filtro_e_salvar(nome_arquivo, janela)

In [17]:
# Compacta tudo
caminho_zip = os.path.join("/content", f"{movimento}_resultados.zip")
with zipfile.ZipFile(caminho_zip, "w") as zipf:
    for raiz, _, arquivos in os.walk(pasta_base):
        for nome_arquivo in arquivos:
            caminho_completo = os.path.join(raiz, nome_arquivo)
            caminho_relativo = os.path.relpath(caminho_completo, start="/content")
            zipf.write(caminho_completo, arcname=caminho_relativo)

In [18]:
# Download ZIP
files.download(caminho_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>